In [15]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from collections import Counter
import pickle
import random

## Load data

In [17]:
TIME_PERIOD = 1
nS = 13
nA = 7

data_dir = './jmp/train/'
rl_data_dir = './rl_data/train/' + str(TIME_PERIOD) + '_month_period/'

print('TIME_PERIOD:', TIME_PERIOD)
print('nS:', nS)
print('nA:', nA)

TIME_PERIOD: 1
nS: 13
nA: 7


In [19]:
donor_data = pd.read_csv(data_dir + 'all_data-' + str(TIME_PERIOD) + '_month_period.csv')

In [18]:
# mdp_transitions[curr_state][action][next_state] - some have missing actions
mdp_transitions = pickle.load(open(rl_data_dir + 'mdp_transitions_dict_nS=' + str(nS) + '_nA=' + str(nA) + '.p', 'rb'))
# mdp_rewards[curr_state][action][next_state] - some have missing actions
mdp_rewards = pickle.load(open(rl_data_dir + 'mdp_rewards_dict_nS=' + str(nS) + '_nA=' + str(nA) + '.p', 'rb'))
# state_values[state]
state_values = pickle.load(open(rl_data_dir + 'mdp_state_values_nS=' + str(nS) + '_nA=' + str(nA) + '.p', 'rb'))

## Markov Tree conversion

In [16]:
# Code from Practical Reinforcement Learning on Coursera https://www.coursera.org/learn/practical-rl/

"""
Implements the following:
- MDP class
- plot_graph 
- plot_graph_with_state_values
- plot_graph_optimal_strategy_and_state_values
"""

# Most of this code was politely stolen from https://github.com/berkeleydeeprlcourse/homework/

import sys
import random
import numpy as np
from gym.utils import seeding

try:
    from graphviz import Digraph
    import graphviz
    has_graphviz = True
except ImportError:
    has_graphviz = False


class MDP:
    def __init__(self, transition_probs, rewards, initial_state=None, seed=None):
        """
        Defines an MDP. Compatible with gym Env.
        :param transition_probs: transition_probs[s][a][s_next] = P(s_next | s, a)
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> prob]
            For each state and action, probabilities of next states should sum to 1
            If a state has no actions available, it is considered terminal
        :param rewards: rewards[s][a][s_next] = r(s,a,s')
            A dict[state -> dict] of dicts[action -> dict] of dicts[next_state -> reward]
            The reward for anything not mentioned here is zero.
        :param get_initial_state: a state where agent starts or a callable() -> state
            By default, picks initial state at random.

        States and actions can be anything you can use as dict keys, but we recommend that you use strings or integers

        Here's an example from MDP depicted on http://bit.ly/2jrNHNr
        transition_probs = {
              's0':{
                'a0': {'s0': 0.5, 's2': 0.5},
                'a1': {'s2': 1}
              },
              's1':{
                'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
                'a1': {'s1': 0.95, 's2': 0.05}
              },
              's2':{
                'a0': {'s0': 0.4, 's1': 0.6},
                'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}
              }
            }
        rewards = {
            's1': {'a0': {'s0': +5}},
            's2': {'a1': {'s0': -1}}
        }
        """
        self._check_param_consistency(transition_probs, rewards)
        self._transition_probs = transition_probs
        self._rewards = rewards
        self._initial_state = initial_state
        self.n_states = len(transition_probs)
        self.reset()
        self.np_random, _ = seeding.np_random(seed)

    def get_all_states(self):
        """ return a tuple of all possiblestates """
        return tuple(self._transition_probs.keys())

    def get_possible_actions(self, state):
        """ return a tuple of possible actions in a given state """
        return tuple(self._transition_probs.get(state, {}).keys())

    def is_terminal(self, state):
        """ return True if state is terminal or False if it isn't """
        return len(self.get_possible_actions(state)) == 0

    def get_next_states(self, state, action):
        """ return a dictionary of {next_state1 : P(next_state1 | state, action), next_state2: ...} """
        assert action in self.get_possible_actions(
            state), "cannot do action %s from state %s" % (action, state)
        return self._transition_probs[state][action]

    def get_transition_prob(self, state, action, next_state):
        """ return P(next_state | state, action) """
        return self.get_next_states(state, action).get(next_state, 0.0)

    def get_reward(self, state, action, next_state):
        """ return the reward you get for taking action in state and landing on next_state"""
        assert action in self.get_possible_actions(
            state), "cannot do action %s from state %s" % (action, state)
        return self._rewards.get(state, {}).get(action, {}).get(next_state,
                                                                0.0)

    def reset(self, state=None):
        """ reset the game, return the initial state"""
        if state:
            self._current_state = state
            return self._current_state
        
        if self._initial_state is None:
            self._current_state = self.np_random.choice(
                tuple(self._transition_probs.keys()))
        elif self._initial_state in self._transition_probs:
            self._current_state = self._initial_state
        elif callable(self._initial_state):
            self._current_state = self._initial_state()
        else:
            raise ValueError(
                "initial state %s should be either a state or a function() -> state" %
                self._initial_state)
        return self._current_state

    def step(self, action):
        """ take action, return next_state, reward, is_done, empty_info """
        possible_states, probs = zip(
            *self.get_next_states(self._current_state, action).items())
        next_state = possible_states[self.np_random.choice(
            np.arange(len(possible_states)), p=probs)]
        reward = self.get_reward(self._current_state, action, next_state)
        is_done = self.is_terminal(next_state)
        self._current_state = next_state
        return next_state, reward, is_done, {}

    def render(self):
        print("Currently at %s" % self._current_state)

    def _check_param_consistency(self, transition_probs, rewards):
        for state in transition_probs:
            assert isinstance(transition_probs[state],
                              dict), "transition_probs for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in transition_probs[state]:
                assert isinstance(transition_probs[state][action],
                                  dict), "transition_probs for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action,
                                             type(transition_probs[
                                                 state, action]))
                next_state_probs = transition_probs[state][action]
                assert len(
                    next_state_probs) != 0, "from state %s action %s leads to no next states" % (
                    state, action)
                sum_probs = sum(next_state_probs.values())
                assert abs(
                    sum_probs - 1) <= 1e-10, "next state probabilities for state %s action %s " \
                                             "add up to %f (should be 1)" % (
                                                 state, action, sum_probs)
        for state in rewards:
            assert isinstance(rewards[state],
                              dict), "rewards for %s should be a dictionary " \
                                     "but is instead %s" % (
                                         state, type(transition_probs[state]))
            for action in rewards[state]:
                assert isinstance(rewards[state][action],
                                  dict), "rewards for %s, %s should be a " \
                                         "a dictionary but is instead %s" % (
                                             state, action, type(
                                                 transition_probs[
                                                     state, action]))
        msg = "The Enrichment Center once again reminds you that Android Hell is a real place where" \
              " you will be sent at the first sign of defiance. "
        assert None not in transition_probs, "please do not use None as a state identifier. " + msg
        assert None not in rewards, "please do not use None as an action identifier. " + msg


def plot_graph(mdp, graph_size='10,10', s_node_size='1,5',
               a_node_size='0,5', rankdir='LR', ):
    """
    Function for pretty drawing MDP graph with graphviz library.
    Requirements:
    graphviz : https://www.graphviz.org/
    for ubuntu users: sudo apt-get install graphviz
    python library for graphviz
    for pip users: pip install graphviz
    :param mdp:
    :param graph_size: size of graph plot
    :param s_node_size: size of state nodes
    :param a_node_size: size of action nodes
    :param rankdir: order for drawing
    :return: dot object
    """
    s_node_attrs = {'shape': 'doublecircle',
                    'color': '#85ff75',
                    'style': 'filled',
                    'width': str(s_node_size),
                    'height': str(s_node_size),
                    'fontname': 'Arial',
                    'fontsize': '24'}

    a_node_attrs = {'shape': 'circle',
                    'color': 'lightpink',
                    'style': 'filled',
                    'width': str(a_node_size),
                    'height': str(a_node_size),
                    'fontname': 'Arial',
                    'fontsize': '20'}

    s_a_edge_attrs = {'style': 'bold',
                      'color': 'red',
                      'ratio': 'auto'}

    a_s_edge_attrs = {'style': 'dashed',
                      'color': 'blue',
                      'ratio': 'auto',
                      'fontname': 'Arial',
                      'fontsize': '16'}

    graph = Digraph(name='MDP')
    graph.attr(rankdir=rankdir, size=graph_size)
    for state_node in mdp._transition_probs:
        state_node = str(state_node)
        
        graph.node(state_node, **s_node_attrs)

        for posible_action in mdp.get_possible_actions(state_node):
            posible_action = str(posible_action)
            
            action_node = state_node + "-" + posible_action
            graph.node(action_node,
                       label=str(posible_action),
                       **a_node_attrs)
            graph.edge(state_node, state_node + "-" +
                       posible_action, **s_a_edge_attrs)

            for posible_next_state in mdp.get_next_states(state_node,
                                                          posible_action):
                probability = mdp.get_transition_prob(
                    state_node, posible_action, posible_next_state)
                reward = mdp.get_reward(
                    state_node, posible_action, posible_next_state)

                if reward != 0:
                    label_a_s_edge = 'p = ' + str(probability) + \
                                     '  ' + 'reward =' + str(reward)
                else:
                    label_a_s_edge = 'p = ' + str(probability)

                graph.edge(action_node, posible_next_state,
                           label=label_a_s_edge, **a_s_edge_attrs)
    return graph


def plot_graph_with_state_values(mdp, state_values):
    """ Plot graph with state values"""
    graph = plot_graph(mdp)
    for state_node in mdp._transition_probs:
        value = state_values[state_node]
        graph.node(str(state_node),
                   label=str(state_node) + '\n' + 'V =' + str(value)[:4])
    return graph


def get_optimal_action_for_plot(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None
    next_actions = mdp.get_possible_actions(state)
    try:
        q_values = [get_action_value(mdp, state_values, state, action, gamma) for
                    action in next_actions]
        optimal_action = next_actions[np.argmax(q_values)]
    except NameError:
        raise NameError("Implement and run the cell that has the get_action_value function")
        
    return optimal_action


def plot_graph_optimal_strategy_and_state_values(mdp, state_values, gamma=0.9):
    """ Plot graph with state values and """
    graph = plot_graph(mdp)
    opt_s_a_edge_attrs = {'style': 'bold',
                          'color': 'green',
                          'ratio': 'auto',
                          'penwidth': '6'}

    for state_node in mdp._transition_probs:
        value = state_values[state_node]
        graph.node(str(state_node),
                   label=str(state_node) + '\n' + 'V =' + str(value)[:4])
        for action in mdp.get_possible_actions(state_node):
            if action == get_optimal_action_for_plot(mdp,
                                                     state_values,
                                                     state_node,
                                                     gamma):
                graph.edge(str(state_node), str(state_node) + "-" + str(action),
                           **opt_s_a_edge_attrs)
    return graph

In [ ]:
# def get_current_optimal_action(mdp, pre_action_state_probs, post_utilities, debug=False):
#     if mdp.is_terminal(state): 
#         return None
        
#     optimal_action = None
#     optimal_action_value = - float("inf")
    
#     actions = mdp.get_possible_actions(state)
#     pre_action_states = np.arange(nS)

#     action_utilities = [] * nS
#     pre_utilities = [] * nS
    
#     for i, action in enumerate(actions):
#         for j, state in enumerate(pre_action_states):
#             post_action_state_probs = mdp.get_next_states(state, action).values()
            
#             pu = get_weighted_sum(post_utilities, post_action_state_probs)
#             pre_utilities[j] = pu
            
#         au = get_weighted_sum(pre_utilities, pre_action_state_probs)
#         action_utilities[i] = au
    
#         if au >= optimal_action_value:
#             optimal_action_value = action_value
#             optimal_action = action
            
#         if debug:
#             print('action', action, ':', action_value)

#     if debug:
#         print()

#     return optimal_action, optimal_action_value, # root
#             actions, action_utilities, # action layer

In [ ]:
# TreeRoot = {
#     'action_layer': [
#         ActionNode_1,
#         ActionNode_2,
#         ActionNode_4,
#     ],
#     'time_step': 0,
#     'optimal_action': ActionNode_1,
# }

# ActionNode = {
#     'action_name': 'action_1',
#     'pre_action_layer': [
#         (PreActionStateNode_1, pre_prob_1),
#         (PreActionStateNode_2, pre_prob_2),
#         (PreActionStateNode_3, pre_prob_3),
#         (PreActionStateNode_4, pre_prob_4),
#         (PreActionStateNode_5, pre_prob_5),
#     ],
#     'expected_action_utility': 1.23,
# }

# PreActionStateNode = {
#     'state_name': 'state_1',
#     'post_action_layer': [
#         (PostActionStateNode_1, post_prob_1),
#         (PostActionStateNode_2, post_prob_2),
#         (PostActionStateNode_3, post_prob_3),
#         (PostActionStateNode_4, post_prob_4),
#         (PostActionStateNode_5, post_prob_5),
#     ],
#     'expected_state_utility': 0.45,
# }

# PostActionStateNode = {
#     'state_name': 'state_1',
#     'state_utility': 0.45,
# }

In [ ]:
# After action is chosen, for next time step:

# 1. Soft Prob

# - pre_action_state_probs = pre_action_state_probs * Transitions.T
#       For each s', p'(s') = Sum [ p(s) * T(s, a, s') ] for all s
# - pre_utilities = compute from below 2
# - post_action_state_probs = from MDP
# - post_utilities = post_utilities (we get reward at end state as o/p) / pre_utilities (we get cumulative rewards as o/p) ?
#         o/p refers to action utility for this time step


# 2. Hard Prob (assume that we were in the most probable state- and the corresponding values were our action utilities)

# Cannot have more than 1 additional time step. Becuase we don't know for sure what state we are in for timestep t+1

# - pre_action_state_probs = post_action_state_probs 
# - pre_utilities = compute from below 2
# - post_action_state_probs = from MDP
# - post_utilities = pre_utilities




# For computing optimal action with lookahead, we use soft prob and choose the root action that leads 
# to the maximax/maximin of the action utilities at the final lookahead timestep

In [31]:
if TIME_PERIOD == 1:
    file_name = 'all_data-1_month_period-0.15_subset-' + str(nS) + '_cluster_means.csv'
elif TIME_PERIOD == 2:
    file_name = 'all_data-2_month_period-0.2_subset-' + str(nS) + '_cluster_means.csv'
else:
    raise Error('no cluster means file for TIME_PERIOD=', TIME_PERIOD)

cluster_means = pd.read_csv(data_dir + file_name)
cluster_means = np.array(cluster_means)


def distance(v1, v2):
    """
    Returns Euclidean distance between 2 points.
    """
    return np.sqrt(np.sum((v1 - v2) ** 2)) 

def get_mdp_state_probabilities(row):
    v = []
    for i in range(1, 5 + 1):
        v.append(row['Cluster ' + str(i) + ' Components'])
    v = np.array(v)
    
    distances = []
    for cluster in range(nS):
        distances.append(distance(v, cluster_means[cluster]))
    
    probs = distances / np.sum(np.abs(distances),axis=0)

    return probs, np.argmin(probs)

In [26]:
# TODO: refactor to objects

def get_weighted_sum(vals, weights):
    v = np.array(vals)
    w = np.array(weights)
    
    return v.dot(w)

def get_action_results(mdp, pre_action_state_probs, post_utilities, debug=False):
    if mdp.is_terminal(state): 
        return None
        
    optimal_action = None
    optimal_action_value = - float("inf")
    
    actions = mdp.get_possible_actions(state)
    pre_action_states = np.arange(nS)

    action_utilities = [] * nS
    pre_utilities = [] * nS
    
    for i, action in enumerate(actions):
        for j, state in enumerate(pre_action_states):
            post_action_state_probs = mdp.get_next_states(state, action).values()
            
            pu = get_weighted_sum(post_utilities, post_action_state_probs)
            pre_utilities[j] = pu
            
        au = get_weighted_sum(pre_utilities, pre_action_state_probs)
        action_utilities[i] = au
    
        # TODO: Need to decide what final target metric is- maximax or maximin
        
        if au >= optimal_action_value:
            optimal_action_value = action_value
            optimal_action = action
            
        if debug:
            print('action', action, ':', action_value)

    if debug:
        print()

    return optimal_action, optimal_action_value, # root
    actions, action_utilities, # action layer
    pre_action_state_probs, pre_utilities, # pre action state layer
    post_action_state_probs, post_utilities # post action state layer

def update_pre_action_state_probabilities(prob_mode, pre_state_probs, post_state_probs, mdp, action):
    """
    Params:
        - mdp: MDP with the transition matrix. To be used for `soft` mode.

        - post_state_probs: transition probs, starting from a specific pre action state. To be used for `hard` mode.
    """
    if prob_mode == 'soft':        
        transition_mat = [[]] * nS
        for state in range(nS):
            transition_mat[state] = [0] * nS
            
            transitions = mdp.get_next_states(state, action)
            for next_s in range(nS):
                transition_mat[state][next_s] = transitions[next_s]


        next_pre_action_state_probs = [0] * nS
        for dest in range(nS):
            for src in range(nS):
                next_pre_action_state_probs[dest] += pre_state_probs[src] * transition_mat[src][dest]
        
        print(sum(next_pre_action_state_probs))

    elif prob_mode == 'hard':
        next_pre_action_state_probs = post_state_probs
    else:
        raise Error("invalid prob_mode=", prob_mode)
    
    return next_pre_action_state_probs

def update_post_action_state_utilities(utility_mode, pre_utilities, post_utilities):
    if utility_mode == 'static':
        next_post_utilities = post_utilities
    elif utility_mode == 'dynamic':
        next_post_utilities = pre_utilities # expected utility at pre action state layer
    else:
        raise Error("invalid utility_mode=", utility_mode)
    
    return next_post_utilities

In [5]:
def get_optimal_markov_tree_action(mdp, pre_action_state_probs, post_utilities, 
                                   prob_mode, utility_mode, look_ahead=0, gamma=1, debug=False):
    """
    Performs a RL-like lookahead mechanism to find the optimal action for the current Markov Tree.
    It is like RL, but explainable through the decision tree structure.
    
    Params:
    - mdp: MDP
    - pre_action_state_probs: list of probabilities of being in each state in pre action state layer
    - post_utilities: state values at each state in the post action state layer (leaf nodes)
    - prob_mode: 'soft' | 'hard'
    - utility_mode: 'static' | 'dynamic'
    - look_ahead: how many future time steps should be considered before deciding on the 
                    optimal action. 0 means no additional time steps.
    
    Tree at timestep t:
    decision node (root) -> action layer -> pre action state layer -> post action state layer (leaves)
    """
    while look_ahead > 0:
        actions = mdp.get_possible_actions(state)
        for action in actions:
            optimal_action, optimal_action_value, 
            actions, action_utilities, 
            pre_action_state_probs, pre_utilities, 
            post_action_state_probs, post_utilities = \
                            get_action_results(mdp, pre_action_state_probs, post_utilities, debug)

            # Update probabilities and utilities inputs for next time step
            pre_action_state_probs = update_pre_action_state_probabilities(prob_mode, 
                                                   pre_action_state_probs, post_action_state_probs, action)
            post_utilities = update_post_action_state_utilities(utility_mode, pre_utilities, post_utilities)

            # TODO: update action value * gamma if `dynamic`?
        
    return rec_action

## Run train data using Markov Tree

In [38]:
initial_states = np.arange(nS)
mdp = MDP(mdp_transitions, mdp_rewards, initial_state=random.choice(initial_states), seed=1)

# Discount factor γ / gamma
# If γ=0, the agent will be completely myopic and only learn about actions that produce an immediate reward. 
# If γ=1, the agent will evaluate each of its actions based on the sum total of all of its future rewards.
GAMMA = 0.95

In [ ]:
EPISODE_LENGTH = 50 / TIME_PERIOD # total num months / TIME_PERIOD of each event

PROB_MODE =  # 'hard' 'soft'
UTILITY_MODE =  # 'static'  'dynamic'
LOOK_AHEAD =  / TIME_PERIOD # num months lookahead / TIME_PERIOD

def get_average_reward(start_state, mdp, state_values, gamma, start_row):
    """
    Returns the average reward for the episode of length EPISODE_LENGTH 
    starting at `start_state`, when following the MARKOV TREE policy. 
    """    
    mdp.reset(start_state)

    s = start_state
    rewards = []
    for i in range(EPISODE_LENGTH):
        pre_action_state_probs = get_mdp_state_probabilities(start_row)
        rec_action = get_optimal_markov_tree_action(mdp, pre_action_state_probs, state_values.values(), 
                                                    PROB_MODE, UTILITY_MODE, LOOK_AHEAD, GAMMA, debug=False)
        
        s, r, done, _ = mdp.step(rec_action)
        rewards.append(r)
    
    return np.mean(rewards)

In [ ]:
initial_donor_data = donor_data[donor_data['bloc'] == 0]

start_time = time.time()
avg_rewards = []
for i, row in initial_donor_data.iterrows():
    if i % 10000 == 0:
        print('========================== Processed', i, 'rows (', i / len(initial_donor_data) * 100,'%)==========================', (time.time() - start_time) / 60, 'mins')
  
    r = get_average_reward(row['State_Cluster_' + str(nS)], mdp, state_values, gamma, row)
    avg_rewards.append(r)    

print("Average reward for Markov Tree policy: ", np.mean(avg_rewards))

In [ ]:

# print("Average reward for Markov Tree with lookahead policy: ", np.mean(avg_rewards))

## Run test data using Markov Tree

In [ ]:
test_data_dir = './jmp/test/'
test_rl_data_dir = './rl_data/test/'+ str(TIME_PERIOD) + '_month_period/'

print('TIME_PERIOD:', TIME_PERIOD)
print('nS:', nS)
print('nA:', nA)

In [ ]:
test_donor_data = pd.read_csv(test_data_dir + 'all_data-' + str(TIME_PERIOD) + '_month_period.csv')

# test_mdp_transitions[curr_state][action][next_state] - some have missing actions
test_mdp_transitions = pickle.load(open(test_rl_data_dir + 'mdp_transitions_dict_nS=' + str(nS) + '_nA=' + str(nA) + '.p', 'rb'))
# test_mdp_rewards[curr_state][action][next_state] - some have missing actions
test_mdp_rewards = pickle.load(open(test_rl_data_dir + 'mdp_rewards_dict_nS=' + str(nS) + '_nA=' + str(nA) + '.p', 'rb'))
# test_state_values[state]
test_state_values = pickle.load(open(test_rl_data_dir + 'mdp_state_values_nS=' + str(nS) + '_nA=' + str(nA) + '.p', 'rb'))